# Monte-Carlo Estimate of $\pi$

We want to estimate the number $\pi$ using a [Monte-Carlo method](https://en.wikipedia.org/wiki/Pi#Monte_Carlo_methods) exploiting that the area of a quarter circle of unit radius is $\pi/4$ and that hence the probability of any randomly chosen point in a unit square to lie in a unit circle centerd at a corner of the unit square is $\pi/4$ as well.  So for N randomly chosen pairs $(x, y)$ with $x\in[0, 1)$ and $y\in[0, 1)$, we count the number $N_{circ}$ of pairs that also satisfy $(x^2 + y^2) < 1$ and estimage $\pi \approx 4 \cdot N_{circ} / N$.

[<img src="https://upload.wikimedia.org/wikipedia/commons/8/84/Pi_30K.gif" 
     width="50%" 
     align=top
     alt="PI monte-carlo estimate">](https://en.wikipedia.org/wiki/Pi#Monte_Carlo_methods)

## The Monte Carlo estimate with Numpy

In [1]:
import numpy as np

In [2]:
N_xy = 10_000_000

xy = np.random.uniform(
    0, 1,
    size=(N_xy, 2)
)

in_circle = ((xy ** 2).sum(axis=-1) < 1)

pi = 4 * in_circle.mean()

print(f"from {xy.nbytes / 1e9} GB of data:")
print(f"pi estimate: {pi}")
print(f"pi error: {abs(pi - np.pi)}")

from 0.16 GB of data:
pi estimate: 3.1417056
pi error: 0.0001129464102067601


We won't be able to scale this to Gigabytes or Terabytes.

## Use Dask Array

Here, we'll see an application of the following Dask basics:

- `LocalCluster`, `Client`, and `dask.array`
- Scaling (local) clusters
- Adaptive (local) clusters

## Set up a local cluster

In [3]:
from dask.distributed import LocalCluster, Client

# Create Dask cluster
cluster = LocalCluster(
    n_workers=2, threads_per_worker=4, memory_limit=4e9,
    ip="0.0.0.0"
)

# attach client (living in this notebook) to cluster
client = Client(cluster)
client

Client Scheduler: tcp://192.168.21.81:39936 Dashboard: http://192.168.21.81:8787/status,Cluster Workers: 2 Cores: 8 Memory: 8.00 GB


## The Monte Carlo Method

In [4]:
import dask.array as darr

def calc_pi_mc(size_in_bytes=100_000_000, number_of_chunks=1_000):
    """Calculate PI using a Monte Carlo estimate."""
    
    array_size = (int(size_in_bytes / 8 / 2), 2)
    chunk_size = (int(array_size[0] / number_of_chunks), 2)
    
    xy = darr.random.uniform(
        0, 1,
        size=array_size,
        chunks=chunk_size
    )
    
    in_circle = ((xy ** 2).sum(axis=-1) < 1)

    pi = 4 * in_circle.mean()
    pi = pi.compute()

    print(f"\nfrom {xy.nbytes / 1e9} GB of data:")
    print(f"\tpi estimate: {pi}")
    print(f"\tpi error: {abs(pi - np.pi)}")
    
    return pi

## The actual calculations

We loop over different volumes of double-precision random numbers and estimate $\pi$ as described above.

In [5]:
%time pi = calc_pi_mc(100_000_000);  # 100 MB

%time pi = calc_pi_mc(1_000_000_000);  # 1 GB

%time pi = calc_pi_mc(10_000_000_000);  # 10 GB

%time pi = calc_pi_mc(100_000_000_000, number_of_chunks=500);  # 100 GB


from 0.1 GB of data:
	pi estimate: 3.14240576
	pi error: 0.0008131064102068208
CPU times: user 1.15 s, sys: 81 ms, total: 1.23 s
Wall time: 1.72 s

from 1.0 GB of data:
	pi estimate: 3.1415376
	pi error: 5.5053589793185864e-05
CPU times: user 1.32 s, sys: 71.8 ms, total: 1.39 s
Wall time: 1.89 s

from 10.0 GB of data:
	pi estimate: 3.1415026176
	pi error: 9.003598979306915e-05
CPU times: user 2 s, sys: 165 ms, total: 2.17 s
Wall time: 5.15 s

from 100.0 GB of data:
	pi estimate: 3.1415837344
	pi error: 8.919189792955251e-06
CPU times: user 3.43 s, sys: 274 ms, total: 3.7 s
Wall time: 35.8 s


## Complete listing of software used here

In [6]:
%pip list

Package          Version            
---------------- -------------------
asciitree        0.3.3              
backcall         0.1.0              
bokeh            1.4.0              
certifi          2019.11.28         
Click            7.0                
cloudpickle      1.3.0              
cycler           0.10.0             
cytoolz          0.10.1             
dask             2.11.0             
dask-jobqueue    0.7.0              
decorator        4.4.1              
distributed      2.11.0             
entrypoints      0.3                
fasteners        0.14.1             
fsspec           0.6.2              
HeapDict         1.0.1              
ipykernel        5.1.4              
ipython          7.12.0             
ipython-genutils 0.2.0              
jedi             0.16.0             
Jinja2           2.11.1             
jupyter-client   6.0.0              
jupyter-core     4.6.3              
kiwisolver       1.1.0              
locket           0.2.0              
M

In [7]:
%conda list --explicit

# This file may be used to create an environment using:
# $ conda create --name <env> --file <this file>
# platform: linux-64
@EXPLICIT
https://conda.anaconda.org/conda-forge/linux-64/_libgcc_mutex-0.1-conda_forge.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/ca-certificates-2019.11.28-hecc5488_0.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/ld_impl_linux-64-2.33.1-h53a641e_8.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/libgfortran-ng-7.3.0-hdf63c60_5.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/libstdcxx-ng-9.2.0-hdf63c60_2.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/llvm-openmp-9.0.1-hc9558a2_2.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/_openmp_mutex-4.5-1_llvm.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/libgcc-ng-9.2.0-h24d8f2e_2.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/expat-2.2.9-he1b5a44_2.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/icu-64.2-he1b5a44_1.tar.bz2
https://conda.a